In [1]:
import numpy as np
import pandas as pd
import math as m

In [2]:
UserRatings = pd.read_csv('UserFeature/user_features.csv',index_col = 0)
UserRatings.columns.name = 'Feature'
UserRatings.head()

Feature,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
User,,,,,,,,,,,,,,,,,,,,,
0,-0.023850,-0.010317,0.043563,0.033008,-0.008568,-0.062246,-0.032498,0.065519,-0.022356,-0.087996,...,-0.045332,0.019596,-0.004090,-0.008276,0.008428,0.009293,-0.039601,0.063574,-0.047043,0.065100
1,0.004085,0.001533,0.009338,-0.004281,0.003913,-0.001472,0.002720,0.014102,-0.000041,-0.006346,...,0.000233,0.003945,0.001471,0.000503,0.004477,0.000182,-0.002312,-0.001367,0.004165,-0.001497
2,-0.000219,0.027465,-0.013980,-0.005643,0.012661,-0.012984,0.008223,-0.003481,-0.017689,0.022440,...,0.008264,0.030386,0.006232,0.007298,0.018440,-0.011802,-0.002255,-0.007778,0.007252,-0.011537
3,-0.132685,0.040978,0.054601,-0.244933,-0.088854,-0.155847,0.030727,-0.030349,0.077923,-0.004483,...,-0.006016,0.018584,0.000894,-0.046703,0.015575,-0.010407,0.005785,0.017658,0.030165,-0.022175
4,0.025887,-0.005396,0.021183,-0.000720,-0.004422,0.007147,0.011078,0.020184,-0.007103,-0.003843,...,0.001725,0.005426,0.004276,-0.002932,0.002457,0.005183,0.005671,0.000756,0.001015,-0.005952


In [3]:
userID = UserRatings.index
n,m = UserRatings.shape
print(n,m)

610 50


In [4]:
def intersection(a,b):
    a = a.dropna()
    b = b.dropna()
    s = tuple(pd.Series(np.intersect1d(a.keys(),b.keys())))
    return s

In [5]:
def union(bookRatingsI, bookRatingsJ):
    first = bookRatingsI.dropna()
    second = bookRatingsJ.dropna()

    bkI = set(first.keys())
    bkJ = set(second.keys())

    b = bkI.union(bkJ)

    # print(len(bkI), len(bkJ), len(b))
    return len(b)

In [18]:
Lathia_Trust = np.zeros((n,n))
Donovan_Trust = np.zeros((n,n))
Shambour_Trust = np.zeros((n,n))
BediSharma_Trust = np.zeros((n,n))
Average_Trust = np.zeros((n,n))
maxRateNorm = UserRatings.max().max()

for i in range(n):
    if i%10==0: print(i)
    Lathia_Trust[i][i]=1
    Donovan_Trust[i][i] = 1
    Shambour_Trust[i][i] = 1
    BediSharma_Trust[i][i] = 1
    Average_Trust[i][i] = 1
    trust,totalTrust=0,0
    
    bookRatingsI =  UserRatings.xs((userID[i]))
    for j in range(i+1,n):
        bookRatingsJ =  UserRatings.xs((userID[j]))
        
        #Lathia et all trust
        d = 1 - (abs(bookRatingsI-bookRatingsJ))/maxRateNorm
        d = d.dropna()
        if (len(d)!= 0):
            trust = sum(d) / len(d)
        # print(trust)
        Lathia_Trust[i][j] = trust
        Lathia_Trust[j][i] = trust
        totalTrust += trust
        trust = 0
        
        #Donovan and Smyth trust
        avgu = bookRatingsI.mean(axis = 0)
        avgv = bookRatingsJ.mean(axis = 0)
        correct = 0
        total = 0
        totalrating = 0
        s = intersection(bookRatingsI, bookRatingsJ)
        for k in s:
            u = float(bookRatingsI[k])
            v = float(bookRatingsJ[k])
            predictedU = avgu + (v - avgv)
            if abs(predictedU-u) <= 0.06:
                correct += 1
        total = len(s)
        if total == 0:
            trust = 0;
        else:
            trust = correct/total
        Donovan_Trust[i][j] = trust
        Donovan_Trust[j][i] = trust
        totalTrust += trust
        trust = 0
        
        #Shambour and Lu trust
        predictedI = avgu + (bookRatingsJ -avgv)
        val = (abs(predictedI-bookRatingsI)/maxRateNorm)**2
        val = val.dropna()
        if len(val)!=0:
            inte = (1 - (sum(val)/len(val)))
            x=len(intersection(bookRatingsI, bookRatingsJ))
            trust = (len(val)/x)*inte
        Shambour_Trust[i][j] = trust
        Shambour_Trust[j][i] = trust
        totalTrust += trust
        trust = 0
        
#       Bedi and Sharma Trust
        k=1
        Ru = bookRatingsI
        Rv = bookRatingsJ
        nmtr = sum(((Ru-avgu)*(Rv-avgv)).dropna())
        dmtr = (sum(((bookRatingsI-avgu)**2).dropna())**(1/2))*(sum(((bookRatingsJ-avgv)**2).dropna())**(1/2))
        iUV = len(intersection(Ru, Rv))
        iU = len(Ru.dropna())
        cUV = iUV/iU
        if dmtr == 0:
            sUV = 0
        else:
            sUV = nmtr/dmtr
        if sUV < 0.0:
            sUV = 0
        if cUV!=0:
            if sUV!=0:
                trust = 2*sUV*cUV / (sUV+cUV)
            else:
                trust = k*cUV
        else:
            trust = 0
        BediSharma_Trust[i][j] = trust
        BediSharma_Trust[j][i] = trust
        totalTrust += trust
        trust = 0
        
        #Average of all types of trust
        Average_Trust[i][j] = Average_Trust[j][i] = totalTrust/4
        totalTrust = 0
        
print(Lathia_Trust)
print('*********')
print(Donovan_Trust)
print('*************')
print(Shambour_Trust)
print('********')
print(BediSharma_Trust)
print('*****************')
print(Average_Trust)

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
[[1.         0.95606526 0.95062397 ... 0.86276309 0.95677733 0.86861642]
 [0.95606526 1.         0.98495171 ... 0.88180117 0.99343779 0.87946228]
 [0.95062397 0.98495171 1.         ... 0.88095797 0.98356174 0.87850385]
 ...
 [0.86276309 0.88180117 0.88095797 ... 1.         0.88213509 0.83882971]
 [0.95677733 0.99343779 0.98356174 ... 0.88213509 1.         0.87754583]
 [0.86861642 0.87946228 0.87850385 ... 0.83882971 0.87754583 1.        ]]
*********
[[1.   0.78 0.78 ... 0.34 0.78 0.32]
 [0.78 1.   1.   ... 0.48 1.   0.42]
 [0.78 1.   1.   ... 0.46 1.   0.36]
 ...
 [0.34 0.48 0.46 ... 1.   0.46 0.34]
 [0.78 1.   1.   ... 0.46 1.   0.42]
 [0.32 0.42 0.36 ... 0.34 0.42 1.  ]]
*************
[[1.         0.99716711 0.99646873 ... 0.96982226 0.99713161 0.97076321

In [19]:
Lathia_Et_al = pd.DataFrame(data = Lathia_Trust, index= userID,columns = userID )
Lathia_Et_al.head(20)

User,0,1,2,3,4,5,6,7,8,9,...,600,601,602,603,604,605,606,607,608,609
User,,,,,,,,,,,,,,,,,,,,,
0,1.000000,0.956065,0.950624,0.933443,0.955311,0.935171,0.947447,0.955610,0.956515,0.951000,...,0.952706,0.956222,0.892977,0.955143,0.955478,0.875663,0.946263,0.862763,0.956777,0.868616
1,0.956065,1.000000,0.984952,0.943062,0.992739,0.961275,0.969691,0.989979,0.991489,0.967012,...,0.981343,0.980449,0.911061,0.989776,0.985194,0.884641,0.972703,0.881801,0.993438,0.879462
2,0.950624,0.984952,1.000000,0.939537,0.982064,0.962612,0.967152,0.982562,0.984542,0.967022,...,0.976033,0.977162,0.912245,0.982741,0.981051,0.877504,0.970624,0.880958,0.983562,0.878504
3,0.933443,0.943062,0.939537,1.000000,0.942147,0.933282,0.945514,0.942708,0.941712,0.945192,...,0.938128,0.940445,0.875795,0.940516,0.944245,0.861921,0.937839,0.869300,0.942816,0.859028
4,0.955311,0.992739,0.982064,0.942147,1.000000,0.958175,0.971257,0.988216,0.988686,0.964925,...,0.980996,0.978609,0.910874,0.988742,0.983612,0.885223,0.968806,0.878676,0.991299,0.876566
5,0.935171,0.961275,0.962612,0.933282,0.958175,1.000000,0.952328,0.958823,0.961947,0.950970,...,0.957294,0.956356,0.893224,0.960771,0.957476,0.870982,0.964310,0.877146,0.958831,0.871500
6,0.947447,0.969691,0.967152,0.945514,0.971257,0.952328,1.000000,0.970407,0.970172,0.954359,...,0.969003,0.967771,0.899472,0.970224,0.971746,0.874785,0.963980,0.868191,0.970577,0.869769
7,0.955610,0.989979,0.982562,0.942708,0.988216,0.958823,0.970407,1.000000,0.991341,0.964855,...,0.984422,0.980718,0.910438,0.989398,0.985379,0.884664,0.972132,0.880570,0.990331,0.877277
8,0.956515,0.991489,0.984542,0.941712,0.988686,0.961947,0.970172,0.991341,1.000000,0.965943,...,0.984067,0.982639,0.913081,0.989018,0.987613,0.885536,0.972573,0.880641,0.990955,0.879362


In [20]:
Donvon_Smyth = pd.DataFrame(data = Donovan_Trust, index= userID,columns = userID )
Donvon_Smyth.head(20)

User,0,1,2,3,4,5,6,7,8,9,...,600,601,602,603,604,605,606,607,608,609
User,,,,,,,,,,,,,,,,,,,,,
0,1.00,0.78,0.78,0.58,0.80,0.64,0.78,0.76,0.78,0.78,...,0.74,0.78,0.42,0.76,0.76,0.42,0.72,0.34,0.78,0.32
1,0.78,1.00,1.00,0.70,1.00,0.84,0.94,1.00,1.00,0.92,...,1.00,1.00,0.56,1.00,1.00,0.44,0.98,0.48,1.00,0.42
2,0.78,1.00,1.00,0.68,1.00,0.80,0.92,1.00,1.00,0.90,...,1.00,0.96,0.64,1.00,1.00,0.36,0.96,0.46,1.00,0.36
3,0.58,0.70,0.68,1.00,0.70,0.66,0.76,0.68,0.68,0.74,...,0.64,0.70,0.32,0.72,0.74,0.40,0.68,0.36,0.70,0.40
4,0.80,1.00,1.00,0.70,1.00,0.82,0.94,1.00,1.00,0.90,...,0.98,1.00,0.60,1.00,1.00,0.46,0.96,0.46,1.00,0.40
5,0.64,0.84,0.80,0.66,0.82,1.00,0.76,0.82,0.82,0.80,...,0.80,0.82,0.42,0.82,0.80,0.38,0.90,0.50,0.82,0.34
6,0.78,0.94,0.92,0.76,0.94,0.76,1.00,0.96,0.96,0.80,...,0.92,0.92,0.46,0.94,0.94,0.44,0.88,0.32,0.94,0.40
7,0.76,1.00,1.00,0.68,1.00,0.82,0.96,1.00,1.00,0.90,...,1.00,1.00,0.56,1.00,1.00,0.46,0.96,0.42,1.00,0.38
8,0.78,1.00,1.00,0.68,1.00,0.82,0.96,1.00,1.00,0.94,...,1.00,1.00,0.60,1.00,1.00,0.46,0.98,0.38,1.00,0.42


In [21]:
Shambour_Lu = pd.DataFrame(data = Shambour_Trust, index= userID,columns = userID )
Shambour_Lu.head(20)

User,0,1,2,3,4,5,6,7,8,9,...,600,601,602,603,604,605,606,607,608,609
User,,,,,,,,,,,,,,,,,,,,,
0,1.000000,0.997167,0.996469,0.992069,0.996978,0.993656,0.995311,0.996973,0.997139,0.995906,...,0.996479,0.997167,0.967136,0.996855,0.996939,0.975061,0.995882,0.969822,0.997132,0.970763
1,0.997167,1.000000,0.999646,0.993339,0.999918,0.997399,0.998465,0.999850,0.999874,0.998182,...,0.999479,0.999385,0.972424,0.999849,0.999669,0.976747,0.998877,0.974044,0.999932,0.975817
2,0.996469,0.999646,1.000000,0.992991,0.999510,0.997269,0.998144,0.999568,0.999696,0.998163,...,0.999217,0.999111,0.973106,0.999550,0.999450,0.973599,0.998804,0.974083,0.999618,0.976041
3,0.992069,0.993339,0.992991,1.000000,0.992965,0.992003,0.993625,0.993620,0.993245,0.993484,...,0.992697,0.992837,0.964635,0.993118,0.993989,0.968107,0.992865,0.967822,0.993521,0.967919
4,0.996978,0.999918,0.999510,0.992965,1.000000,0.997190,0.998606,0.999797,0.999814,0.997981,...,0.999417,0.999267,0.972221,0.999775,0.999571,0.976507,0.998576,0.973554,0.999885,0.974842
5,0.993656,0.997399,0.997269,0.992003,0.997190,1.000000,0.996784,0.997284,0.997484,0.995991,...,0.997196,0.996836,0.969453,0.997427,0.997159,0.973379,0.997899,0.970654,0.997127,0.974089
6,0.995311,0.998465,0.998144,0.993625,0.998606,0.996784,1.000000,0.998688,0.998635,0.996096,...,0.998373,0.998260,0.970962,0.998498,0.998585,0.972737,0.997899,0.971361,0.998557,0.972386
7,0.996973,0.999850,0.999568,0.993620,0.999797,0.997284,0.998688,1.000000,0.999883,0.997972,...,0.999623,0.999367,0.972764,0.999822,0.999710,0.976191,0.998774,0.974461,0.999849,0.974679
8,0.997139,0.999874,0.999696,0.993245,0.999814,0.997484,0.998635,0.999883,1.000000,0.998119,...,0.999604,0.999488,0.972487,0.999814,0.999751,0.976117,0.998835,0.974214,0.999863,0.974948


In [22]:
BediSharma_Trust_Trust_Matrix = pd.DataFrame(data = BediSharma_Trust, index= userID,columns = userID )
BediSharma_Trust_Trust_Matrix.head(20)

User,0,1,2,3,4,5,6,7,8,9,...,600,601,602,603,604,605,606,607,608,609
User,,,,,,,,,,,,,,,,,,,,,
0,1.000000,0.144393,1.000000,0.296878,1.000000,1.000000,1.000000,1.000000,0.083426,0.229422,...,1.000000,0.341764,1.000000,1.000000,0.053915,0.198870,1.000000,1.000000,0.088810,1.000000
1,0.144393,1.000000,0.087232,1.000000,0.739617,1.000000,1.000000,1.000000,1.000000,0.224295,...,1.000000,1.000000,1.000000,0.118442,1.000000,0.498204,1.000000,1.000000,0.582760,0.543285
2,1.000000,0.087232,1.000000,1.000000,1.000000,0.117133,1.000000,1.000000,0.401946,0.350292,...,1.000000,1.000000,0.271822,1.000000,0.110408,1.000000,0.250145,0.053522,0.007280,0.378291
3,0.296878,1.000000,1.000000,1.000000,1.000000,0.242641,0.416425,0.338349,1.000000,0.432917,...,1.000000,1.000000,1.000000,1.000000,0.486892,1.000000,0.176086,0.013485,0.246034,1.000000
4,1.000000,0.739617,1.000000,1.000000,1.000000,1.000000,0.162477,0.040246,1.000000,1.000000,...,1.000000,1.000000,1.000000,0.097007,1.000000,0.274238,1.000000,1.000000,0.592286,1.000000
5,1.000000,1.000000,0.117133,0.242641,1.000000,1.000000,0.300215,1.000000,0.156927,0.154540,...,0.118618,1.000000,1.000000,0.114426,1.000000,1.000000,0.622786,1.000000,1.000000,0.179849
6,1.000000,1.000000,1.000000,0.416425,0.162477,0.300215,1.000000,0.286162,0.100698,1.000000,...,0.149708,0.078436,1.000000,1.000000,0.278102,1.000000,0.214281,1.000000,1.000000,1.000000
7,1.000000,1.000000,1.000000,0.338349,0.040246,1.000000,0.286162,1.000000,0.232897,1.000000,...,0.466148,1.000000,0.209258,0.163098,0.331501,0.127421,1.000000,0.186954,1.000000,1.000000
8,0.083426,1.000000,0.401946,1.000000,1.000000,0.156927,0.100698,0.232897,1.000000,0.049286,...,0.348529,0.018323,0.036267,1.000000,0.455130,0.074829,1.000000,0.018959,1.000000,1.000000


In [23]:
Average_Trust_Matrix = pd.DataFrame(data = Average_Trust, index= userID,columns = userID )
Average_Trust_Matrix.head(20)

User,0,1,2,3,4,5,6,7,8,9,...,600,601,602,603,604,605,606,607,608,609
User,,,,,,,,,,,,,,,,,,,,,
0,1.000000,0.719406,0.931773,0.700598,0.938072,0.892207,0.930690,0.928146,0.704270,0.739082,...,0.922296,0.768788,0.820028,0.928000,0.691583,0.617399,0.915536,0.793146,0.705680,0.789845
1,0.719406,1.000000,0.767957,0.909100,0.933069,0.949669,0.977039,0.997457,0.997841,0.777372,...,0.995206,0.994959,0.860871,0.777017,0.996216,0.699898,0.987895,0.833961,0.894032,0.704641
2,0.931773,0.767957,1.000000,0.903132,0.995394,0.719253,0.971324,0.995532,0.846546,0.803869,...,0.993812,0.984068,0.699293,0.995573,0.772727,0.802776,0.794893,0.592141,0.747615,0.648209
3,0.700598,0.909100,0.903132,1.000000,0.908778,0.706982,0.778891,0.738669,0.903739,0.777898,...,0.892706,0.908320,0.790107,0.913408,0.791282,0.807507,0.696697,0.552652,0.720593,0.806737
4,0.938072,0.933069,0.995394,0.908778,1.000000,0.943841,0.768085,0.757065,0.997125,0.965727,...,0.990103,0.994469,0.870774,0.771381,0.995796,0.648992,0.981845,0.828058,0.895867,0.812852
5,0.892207,0.949669,0.719253,0.706982,0.943841,1.000000,0.752332,0.944027,0.734090,0.725375,...,0.718277,0.943298,0.820669,0.723156,0.938659,0.806090,0.871249,0.836950,0.943990,0.591360
6,0.930690,0.977039,0.971324,0.778891,0.768085,0.752332,1.000000,0.803814,0.757376,0.937614,...,0.759271,0.741117,0.832609,0.977181,0.797108,0.821881,0.764040,0.789888,0.977283,0.810539
7,0.928146,0.997457,0.995532,0.738669,0.757065,0.944027,0.803814,1.000000,0.806030,0.965707,...,0.862548,0.995021,0.663115,0.788080,0.829147,0.612069,0.982727,0.615496,0.997545,0.807989
8,0.704270,0.997841,0.846546,0.903739,0.997125,0.734090,0.757376,0.806030,1.000000,0.738337,...,0.833050,0.750112,0.630459,0.997208,0.860623,0.599120,0.987852,0.563453,0.997704,0.818577


In [24]:
Lathia_Et_al.to_csv('UserFeature/Lathia_Et_al_TrustMatrixUserFeature.csv')
Donvon_Smyth.to_csv('UserFeature/Donvon_Smyth_TrustMatrixFeature.csv')
Shambour_Lu.to_csv('UserFeature/Shambour_Lu_TrustMatrixFeature.csv')
BediSharma_Trust_Trust_Matrix.to_csv('UserFeature/BediSharma_Trust_Trust_MatrixFeature.csv')
Average_Trust_Matrix.to_csv('UserFeature/Average_Trust_MatrixFeature.csv')